This notebook builds from "DECALS low-SB_completeness AnaK overlap"

In [1]:
from __future__ import print_function, division

In [2]:
# This changes the current directory to the base saga directory - make sure to run this first!
# This is necessary to be able to import the py files and use the right directories,
# while keeping all the notebooks in their own directory.
import os
import sys
from time import time

if 'saga_base_dir' not in locals():
    saga_base_dir = os.path.abspath('..')
if saga_base_dir not in sys.path:
    os.chdir(saga_base_dir)

In [3]:
import hosts
import targeting

import numpy as np

from scipy import interpolate

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy import table
from astropy.table import Table
from astropy.io import fits

from astropy.utils.console import ProgressBar

from collections import Counter

In [4]:
%matplotlib inline
from matplotlib import style, pyplot as plt

plt.style.use('seaborn-deep')
plt.rcParams['image.cmap'] = 'viridis'
plt.rcParams['image.origin'] = 'lower'
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['axes.titlesize'] =  plt.rcParams['axes.labelsize'] = 16
plt.rcParams['xtick.labelsize'] =  plt.rcParams['ytick.labelsize'] = 14

In [5]:
from IPython import display

In [6]:
from decals import make_cutout_comparison_table, fluxivar_to_mag_magerr, compute_sb

# Get AnaK/brick info

In [7]:
hsts = hosts.get_saga_hosts_from_google(clientsecretjsonorfn='client_secrets.json', useobservingsummary=False)
anak  = [h for h in hsts if h.name=='AnaK']
assert len(anak)==1
anak = anak[0]

Using cached version of google hosts list from file "hosts_dl.pkl2"


In [8]:
bricknames = []
with open('decals_dr3/anakbricks') as f:
    for l in f:
        l = l.strip()
        if l != '':
            bricknames.append(l)
print(bricknames)

['3538m007', '3541m007', '3543m007', '3546m007', '3533m005', '3536m005', '3538m005', '3541m005', '3543m005', '3546m005', '3548m005', '3531m002', '3533m002', '3536m002', '3538m002', '3541m002', '3543m002', '3546m002', '3548m002', '3551m002', '3531p000', '3533p000', '3536p000', '3538p000', '3541p000', '3543p000', '3546p000', '3548p000', '3551p000', '3531p002', '3533p002', '3536p002', '3538p002', '3541p002', '3543p002', '3546p002', '3548p002', '3551p002', '3531p005', '3533p005', '3536p005', '3538p005', '3541p005', '3543p005', '3546p005', '3548p005', '3551p005', '3531p007', '3533p007', '3536p007', '3538p007', '3541p007', '3543p007', '3546p007', '3548p007', '3551p007', '3533p010', '3536p010', '3538p010', '3541p010', '3543p010', '3546p010', '3548p010', '3536p012', '3538p012', '3541p012', '3543p012', '3546p012']


# Download the catalogs if needed 

In [9]:
base_url = 'http://portal.nersc.gov/project/cosmo/data/legacysurvey/dr3/tractor/{first3}/tractor-{brickname}.fits'

for brickname in ProgressBar(bricknames, ipython_widget=True):
    url = base_url.format(brickname=brickname, first3=brickname[:3])
    target = os.path.join('decals_dr3/catalogs/', url.split('/')[-1])
    if not os.path.isfile(target):
        !wget $url -O $target
    else:
        print(target, 'already exists, not downloading')

decals_dr3/catalogs/tractor-3538m007.fits already exists, not downloading
decals_dr3/catalogs/tractor-3541m007.fits already exists, not downloading
decals_dr3/catalogs/tractor-3543m007.fits already exists, not downloading
decals_dr3/catalogs/tractor-3546m007.fits already exists, not downloading
decals_dr3/catalogs/tractor-3533m005.fits already exists, not downloading
decals_dr3/catalogs/tractor-3536m005.fits already exists, not downloading
decals_dr3/catalogs/tractor-3538m005.fits already exists, not downloading
decals_dr3/catalogs/tractor-3541m005.fits already exists, not downloading
decals_dr3/catalogs/tractor-3543m005.fits already exists, not downloading
decals_dr3/catalogs/tractor-3546m005.fits already exists, not downloading
decals_dr3/catalogs/tractor-3548m005.fits already exists, not downloading
decals_dr3/catalogs/tractor-3531m002.fits already exists, not downloading
decals_dr3/catalogs/tractor-3533m002.fits already exists, not downloading
decals_dr3/catalogs/tractor-3536m002.f

# Load the catalogs 

In [10]:
bricks = Table.read('decals_dr3/survey-bricks.fits.gz')
bricksdr3 = Table.read('decals_dr3/survey-bricks-dr3.fits.gz')

In [11]:
catalog_fns = ['decals_dr3/catalogs/tractor-{}.fits'.format(bnm) for bnm in bricknames]
decals_catalogs = [Table.read(fn) for fn in catalog_fns]
dcatall = table.vstack(decals_catalogs, metadata_conflicts='silent')

In [12]:
sdss_catalog = Table.read('catalogs/base_sql_nsa{}.fits.gz'.format(anak.nsaid))

In [13]:
#cut out the non-overlap region
dsc = SkyCoord(dcatall['ra'], dcatall['dec'], unit=u.deg)
dcutall = dcatall[dsc.separation(anak.coords) < 1*u.deg]

## Populate additional  info in the DECaLS tables

In [14]:
ap_sizes = [0.5,0.75,1.0,1.5,2.0,3.5,5.0,7.0] * u.arcsec 

In [15]:
for dcat in [dcutall]:
    for magnm, idx in zip('grz', [1, 2, 4]):
        mag, mag_err = fluxivar_to_mag_magerr(dcat['decam_flux'][:, 1], dcat['decam_flux_ivar'][:, idx])
        dcat[magnm] = mag
        dcat[magnm + '_err'] = mag_err
    
    dcat['sb_r_0.5'] = compute_sb(0.5*u.arcsec, dcat['decam_apflux'][:, 2, :])
    dcat['sb_r_0.75'] = compute_sb(0.75*u.arcsec, dcat['decam_apflux'][:, 2, :])
    dcat['sb_r_1'] = compute_sb(1.0*u.arcsec, dcat['decam_apflux'][:, 2, :])
    dcat['sb_r_2'] = compute_sb(2.0*u.arcsec, dcat['decam_apflux'][:, 2, :])

/Users/erik/miniconda3/envs/saga/lib/python2.7/site-packages/ipykernel/__main__.py:11: RuntimeWarning: divide by zero encountered in log10
/Users/erik/miniconda3/envs/saga/lib/python2.7/site-packages/ipykernel/__main__.py:11: RuntimeWarning: invalid value encountered in log10
/Users/erik/miniconda3/envs/saga/lib/python2.7/site-packages/ipykernel/__main__.py:12: RuntimeWarning: divide by zero encountered in log10
/Users/erik/miniconda3/envs/saga/lib/python2.7/site-packages/ipykernel/__main__.py:12: RuntimeWarning: invalid value encountered in log10
/Users/erik/miniconda3/envs/saga/lib/python2.7/site-packages/ipykernel/__main__.py:13: RuntimeWarning: divide by zero encountered in log10
/Users/erik/miniconda3/envs/saga/lib/python2.7/site-packages/ipykernel/__main__.py:13: RuntimeWarning: invalid value encountered in log10
/Users/erik/miniconda3/envs/saga/lib/python2.7/site-packages/ipykernel/__main__.py:8: RuntimeWarning: divide by zero encountered in log10
/Users/erik/miniconda3/envs/sag

# Experiment with Masks 

In [16]:
MASK_BITS = {0: 'detector bad pixel/no data',
             1: 'saturated',
             2: 'interpolated',
             4: 'single exposure cosmic ray',
             6: 'bleed trail',
             7: 'multi-exposure transient',
             8: 'edge',
             9: 'edge2',
             10: 'longthin'}
def maskarr_to_boolarrs(arr):
    boolarrs = {}
    for bnum, name in MASK_BITS.items():
        boolarrs[name] = (arr & 10**bnum)!=0
    return boolarrs

In [17]:
anymaskr = maskarr_to_boolarrs(dcutall['decam_anymask'][:, 2])
allmaskr = maskarr_to_boolarrs(dcutall['decam_allmask'][:, 2])

In [18]:
nper = 10

## Any pixels?

In [19]:
for name, barr in anymaskr.items():
    print(name)
    idxs = np.random.permutation(np.sum(barr))[:nper]
    display.display(make_cutout_comparison_table(dcutall[barr][idxs], doprint=False))

longthin


obj,DECALS,SDSS
334664_2110RA=354.1556Dec=0.4542r=25.04sb=25.44anymsk=[ 0 0 1024 0 0 0]allmsk=[0 0 0 0 0 0],,
334663_2274RA=353.9664Dec=0.4609r=24.29sb=25.21anymsk=[ 0 0 1537 0 0 0]allmsk=[0 0 0 0 0 0],,
334665_1589RA=354.3837Dec=0.4417r=24.24sb=24.99anymsk=[ 0 1 1024 0 1 0]allmsk=[0 0 0 0 0 0],,
334665_1564RA=354.4223Dec=0.4401r=24.75sb=25.38anymsk=[ 0 0 1025 0 1 0]allmsk=[0 0 0 0 0 0],,
334665_1593RA=354.3725Dec=0.4417r=24.28sb=25.05anymsk=[ 0 0 1024 0 1 0]allmsk=[0 0 0 0 0 0],,
334664_2167RA=354.1081Dec=0.4549r=24.60sb=25.38anymsk=[ 0 0 1024 0 0 0]allmsk=[0 0 0 0 0 0],,
337543_4576RA=353.7505Dec=1.0493r=23.84sb=24.67anymsk=[ 0 0 1024 0 0 0]allmsk=[0 0 0 0 0 0],,
334664_2309RA=354.0929Dec=0.4608r=24.93sb=25.50anymsk=[ 0 0 1537 0 0 0]allmsk=[0 0 0 0 0 0],,
334665_1733RA=354.4130Dec=0.4490r=18.90sb=19.75anymsk=[ 0 0 1024 0 0 0]allmsk=[0 0 0 0 0 0],,
336108_1486RA=355.0468Dec=0.6854r=23.90sb=25.00anymsk=[ 0 0 1024 0 0 0]allmsk=[0 0 0 0 0 0],,


saturated


obj,DECALS,SDSS
334664_444RA=354.1322Dec=0.3898r=18.48sb=17.82anymsk=[ 0 128 70 0 71 0]allmsk=[0 0 0 0 0 0],,
328906_5866RA=354.5134Dec=-0.3964r=15.12sb=17.04anymsk=[ 0 0 78 0 1094 0]allmsk=[0 0 0 0 0 0],,
336107_3823RA=354.7990Dec=0.7564r=15.32sb=17.23anymsk=[0 0 3 0 3 0]allmsk=[0 0 0 0 0 0],,
330342_3605RA=353.5575Dec=-0.2291r=14.16sb=18.32anymsk=[0 2 2 0 2 0]allmsk=[0 0 2 0 2 0],,
337541_592RA=353.3381Dec=0.8952r=15.43sb=16.91anymsk=[0 0 3 0 2 0]allmsk=[0 0 0 0 2 0],,
331784_5791RA=354.1804Dec=0.0970r=14.66sb=18.69anymsk=[ 0 70 583 0 70 0]allmsk=[0 0 0 0 2 0],,
334663_792RA=353.9962Dec=0.4019r=20.15sb=18.23anymsk=[ 0 0 70 0 70 0]allmsk=[ 0 0 0 0 64 0],,
337546_1684RA=354.6930Dec=0.9431r=15.30sb=16.63anymsk=[ 0 0 201 0 70 0]allmsk=[0 0 0 0 0 0],,
331787_846RA=354.8216Dec=-0.0952r=15.50sb=16.72anymsk=[ 0 0 131 0 70 0]allmsk=[0 0 0 0 0 0],,
328904_2039RA=354.0585Dec=-0.5565r=15.30sb=16.93anymsk=[0 0 2 0 2 0]allmsk=[0 0 0 0 0 0],,


bleed trail


...

obj,DECALS,SDSS
337543_2336RA=353.9508Dec=0.9547r=21.67sb=23.55anymsk=[ 0 0 513 0 0 0]allmsk=[0 0 0 0 0 0],,
331781_2124RA=353.3101Dec=-0.0393r=20.27sb=22.01anymsk=[ 0 0 128 0 1024 0]allmsk=[0 0 0 0 0 0],,
334661_5748RA=353.2612Dec=0.6106r=24.36sb=25.60anymsk=[ 0 0 513 0 0 0]allmsk=[0 0 0 0 0 0],,
334667_2980RA=354.8740Dec=0.4997r=23.92sb=24.68anymsk=[ 0 0 513 0 513 0]allmsk=[0 0 0 0 0 0],,
337544_3166RA=354.2263Dec=0.9942r=24.30sb=24.82anymsk=[ 0 0 513 0 641 0]allmsk=[0 0 0 0 0 0],,
334667_6025RA=354.9932Dec=0.6240r=19.53sb=20.48anymsk=[ 0 0 513 0 0 0]allmsk=[0 0 0 0 0 0],,
334666_2662RA=354.5938Dec=0.4765r=24.18sb=25.10anymsk=[ 0 0 513 0 0 0]allmsk=[0 0 0 0 0 0],,
334666_641RA=354.5734Dec=0.3991r=24.68sb=25.16anymsk=[ 0 0 144 0 0 0]allmsk=[0 0 0 0 0 0],,
331787_917RA=354.7518Dec=-0.0952r=24.17sb=25.15anymsk=[ 0 0 513 0 0 0]allmsk=[0 0 0 0 0 0],,
336102_4950RA=353.6434Dec=0.8139r=24.22sb=25.30anymsk=[ 0 1 513 0 0 0]allmsk=[0 0 0 0 0 0],,


single exposure cosmic ray


obj,DECALS,SDSS
334663_4038RA=353.8489Dec=0.5261r=25.18sb=26.06anymsk=[ 0 0 513 0 0 0]allmsk=[0 0 0 0 0 0],,
337545_4432RA=354.3576Dec=1.0475r=24.58sb=25.27anymsk=[ 0 0 144 0 1 0]allmsk=[0 0 0 0 0 0],,
333225_1750RA=354.3294Dec=0.1830r=26.17sb=26.53anymsk=[ 0 0 16 0 0 0]allmsk=[0 0 0 0 0 0],,
337544_3321RA=354.1438Dec=1.0009r=24.99sb=25.60anymsk=[ 0 1 513 0 513 0]allmsk=[0 0 0 0 0 0],,
337546_2540RA=354.5036Dec=0.9779r=24.64sb=25.81anymsk=[ 0 0 513 0 513 0]allmsk=[0 0 0 0 0 0],,
337544_2212RA=354.0353Dec=0.9547r=22.49sb=23.59anymsk=[ 0 0 513 0 0 0]allmsk=[0 0 0 0 0 0],,
337544_2023RA=354.1209Dec=0.9548r=23.47sb=24.75anymsk=[ 0 0 641 0 0 0]allmsk=[0 0 0 0 0 0],,
336102_111RA=353.7059Dec=0.6289r=24.53sb=25.20anymsk=[ 0 0 16 0 0 0]allmsk=[0 0 0 0 0 0],,
334667_3147RA=354.7651Dec=0.5077r=24.14sb=24.86anymsk=[ 0 0 513 0 513 0]allmsk=[0 0 0 0 0 0],,
331787_2721RA=354.9934Dec=-0.0181r=23.20sb=24.29anymsk=[ 0 0 513 0 0 0]allmsk=[0 0 0 0 0 0],,


detector bad pixel/no data


obj,DECALS,SDSS
333223_2743RA=353.9207Dec=0.2311r=24.91sb=25.51anymsk=[0 0 1 0 0 0]allmsk=[0 0 0 0 0 0],,
334665_520RA=354.2784Dec=0.3874r=23.82sb=25.03anymsk=[0 1 1 0 0 0]allmsk=[0 0 0 0 0 0],,
336102_77RA=353.5544Dec=0.6257r=24.72sb=25.39anymsk=[ 0 0 513 0 0 0]allmsk=[0 0 0 0 0 0],,
330341_3680RA=353.3112Dec=-0.2424r=24.87sb=25.73anymsk=[0 1 1 0 0 0]allmsk=[0 0 0 0 0 0],,
333222_2028RA=353.4965Dec=0.2012r=24.17sb=25.23anymsk=[0 1 1 0 0 0]allmsk=[0 0 0 0 0 0],,
331787_4057RA=354.9309Dec=0.0380r=24.11sb=24.97anymsk=[0 0 1 0 0 0]allmsk=[0 0 0 0 0 0],,
337547_1039RA=354.9154Dec=0.9091r=23.44sb=24.18anymsk=[0 0 1 0 0 0]allmsk=[0 0 0 0 0 0],,
336101_2513RA=353.4076Dec=0.7255r=23.24sb=24.10anymsk=[0 0 1 0 0 0]allmsk=[0 0 0 0 0 0],,
333222_3018RA=353.5203Dec=0.2459r=23.39sb=24.72anymsk=[0 0 1 0 0 0]allmsk=[0 0 0 0 0 0],,
336103_4724RA=353.9329Dec=0.7964r=19.79sb=22.15anymsk=[0 1 1 0 1 0]allmsk=[0 0 0 0 0 0],,


## All pixels?

In [20]:
for name, barr in allmaskr.items():
    print(name)
    idxs = np.random.permutation(np.sum(barr))[:nper]
    display.display(make_cutout_comparison_table(dcutall[barr][idxs], doprint=False))

longthin


obj,DECALS,SDSS


saturated


obj,DECALS,SDSS
331785_1880RA=354.3395Dec=-0.0399r=13.58sb=18.35anymsk=[ 0 70 70 0 70 0]allmsk=[0 0 2 0 2 0],,
333227_520RA=354.9626Dec=0.1528r=13.79sb=18.69anymsk=[0 2 2 0 2 0]allmsk=[0 0 2 0 2 0],,
331782_6085RA=353.6824Dec=0.1302r=10.89sb=17.34anymsk=[0 2 2 0 2 0]allmsk=[0 2 2 0 2 0],,
333225_3155RA=354.4155Dec=0.2426r=11.79sb=17.69anymsk=[0 2 2 0 2 0]allmsk=[0 2 2 0 2 0],,
334663_2052RA=353.8050Dec=0.4546r=14.42sb=18.70anymsk=[ 0 0 70 0 70 0]allmsk=[0 0 2 0 2 0],,
331787_5814RA=354.9261Dec=0.1230r=12.76sb=18.12anymsk=[ 0 70 70 0 70 0]allmsk=[0 2 2 0 2 0],,
336101_4959RA=353.4648Dec=0.8230r=13.77sb=19.28anymsk=[0 2 2 0 2 0]allmsk=[0 0 2 0 2 0],,
337546_5564RA=354.6109Dec=1.1110r=11.82sb=17.57anymsk=[ 0 70 70 0 70 0]allmsk=[0 2 2 0 2 0],,
334663_710RA=353.8068Dec=0.3990r=17.98sb=18.99anymsk=[ 0 64 70 0 70 0]allmsk=[0 0 2 0 2 0],,
334668_21RA=355.0673Dec=0.3692r=14.12sb=17.38anymsk=[0 2 2 0 3 0]allmsk=[0 0 2 0 0 0],,


bleed trail


...

obj,DECALS,SDSS


single exposure cosmic ray


obj,DECALS,SDSS
328904_465RA=354.0317Dec=-0.6150r=23.19sb=24.39anymsk=[ 0 0 16 0 0 0]allmsk=[ 0 0 16 0 0 0],,


detector bad pixel/no data


obj,DECALS,SDSS
328905_549RA=354.3570Dec=-0.6076r=25.00sb=26.29anymsk=[0 0 1 0 0 0]allmsk=[0 0 1 0 0 0],,
